In [ ]:
# https://github.com/gtfintechlab/fomc-hawkish-dovish/blob/main/code_model/bert_fine_tune_lm_hawkish_dovish_train_test.py

In [1]:
import os
import sys
from time import time, sleep
import pandas as pd
from transformers import (BertForSequenceClassification,
                          BertTokenizerFast,
                          RobertaTokenizerFast,
                          RobertaForSequenceClassification,
                          AutoTokenizer,
                          AutoModelForSequenceClassification)
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from sklearn.metrics import f1_score
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification

In [2]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks/Comp545_FinalProject')

Mounted at /content/drive


In [3]:
def train_lm_hawkish_dovish(gpu_numbers: str,
                            train_data_path: str,
                            test_data_path: str,
                            language_model_to_use: str,
                            batch_size: int,
                            learning_rate: float,
                            save_model_path: str):
    """
    Description: Run experiment over particular batch size, learning rate
    """
    # set gpu
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_numbers)
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print("Device assigned: ", device)

    # load training data
    data_df = pd.read_excel(train_data_path)
    sentences = data_df['sentence'].to_list()
    labels = data_df['label'].to_numpy()

    # load test data
    data_df_test = pd.read_excel(test_data_path)
    sentences_test = data_df_test['sentence'].to_list()
    labels_test = data_df_test['label'].to_numpy()

    # load tokenizer
    try:
        if language_model_to_use == 'bert':
            tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', do_lower_case=True, do_basic_tokenize=True)
        elif language_model_to_use == 'roberta':
            tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', do_lower_case=True, do_basic_tokenize=True)
        elif language_model_to_use == 'flangroberta':
            tokenizer = AutoTokenizer.from_pretrained('SALT-NLP/FLANG-Roberta', do_lower_case=True, do_basic_tokenize=True)
        elif language_model_to_use == 'finbert':
            tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone', do_lower_case=True, do_basic_tokenize=True)
        elif language_model_to_use == 'flangbert':
            tokenizer = BertTokenizerFast.from_pretrained('SALT-NLP/FLANG-BERT', do_lower_case=True, do_basic_tokenize=True)
        elif language_model_to_use == 'bert-large':
            tokenizer = BertTokenizerFast.from_pretrained('bert-large-uncased', do_lower_case=True, do_basic_tokenize=True)
        elif language_model_to_use == 'roberta-large':
            tokenizer = RobertaTokenizerFast.from_pretrained('roberta-large', do_lower_case=True, do_basic_tokenize=True)
        else:
            return -1
    except Exception as e:
        print(e)
        sleep(600)
        if language_model_to_use == 'bert':
            tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', do_lower_case=True, do_basic_tokenize=True)
        elif language_model_to_use == 'roberta':
            tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', do_lower_case=True, do_basic_tokenize=True)
        elif language_model_to_use == 'flangroberta':
            tokenizer = AutoTokenizer.from_pretrained('SALT-NLP/FLANG-Roberta', do_lower_case=True, do_basic_tokenize=True)
        elif language_model_to_use == 'finbert':
            tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone', do_lower_case=True, do_basic_tokenize=True)
        elif language_model_to_use == 'flangbert':
            tokenizer = BertTokenizerFast.from_pretrained('SALT-NLP/FLANG-BERT', do_lower_case=True, do_basic_tokenize=True)
        elif language_model_to_use == 'bert-large':
            tokenizer = BertTokenizerFast.from_pretrained('bert-large-uncased', do_lower_case=True, do_basic_tokenize=True)
        elif language_model_to_use == 'roberta-large':
            tokenizer = RobertaTokenizerFast.from_pretrained('roberta-large', do_lower_case=True, do_basic_tokenize=True)
        else:
            return -1

    max_length = 0
    sentence_input = []
    labels_output = []
    for i, sentence in enumerate(sentences):
        if isinstance(sentence, str):
            tokens = tokenizer(sentence)['input_ids']
            sentence_input.append(sentence)
            max_length = max(max_length, len(tokens))
            labels_output.append(labels[i])
        else:
            pass

    max_length=256

    if language_model_to_use == 'flangroberta':
        max_length=128

    tokens = tokenizer(sentence_input, return_tensors='pt', padding=True, truncation=True, max_length=max_length)
    labels = np.array(labels_output)

    input_ids = tokens['input_ids']
    attention_masks = tokens['attention_mask']
    labels = torch.LongTensor(labels)
    dataset = TensorDataset(input_ids, attention_masks, labels)
    val_length = int(len(dataset) * 0.2)
    train_length = len(dataset) - val_length
    print(f'Train Size: {train_length}, Validation Size: {val_length}')
    experiment_results = []

    # select language model
    try:
        if language_model_to_use == 'bert':
            model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3).to(device)
        elif language_model_to_use == 'roberta':
            model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3).to(device)
        elif language_model_to_use == 'flangroberta':
            model = AutoModelForSequenceClassification.from_pretrained('SALT-NLP/FLANG-Roberta', num_labels=3).to(device)
        elif language_model_to_use == 'finbert':
            model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3).to(device)
        elif language_model_to_use == 'flangbert':
            model = BertForSequenceClassification.from_pretrained('SALT-NLP/FLANG-BERT', num_labels=3).to(device)
        elif language_model_to_use == 'bert-large':
            model = BertForSequenceClassification.from_pretrained('bert-large-uncased', num_labels=3).to(device)
        elif language_model_to_use == 'roberta-large':
            model = RobertaForSequenceClassification.from_pretrained('roberta-large', num_labels=3).to(device)
        else:
            return -1
    except:
        sleep(600)
        if language_model_to_use == 'bert':
            model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3).to(device)
        elif language_model_to_use == 'roberta':
            model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3).to(device)
        elif language_model_to_use == 'flangroberta':
            model = AutoModelForSequenceClassification.from_pretrained('SALT-NLP/FLANG-Roberta', num_labels=3).to(device)
        elif language_model_to_use == 'finbert':
            model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3).to(device)
        elif language_model_to_use == 'flangbert':
            model = BertForSequenceClassification.from_pretrained('SALT-NLP/FLANG-BERT', num_labels=3).to(device)
        elif language_model_to_use == 'bert-large':
            model = BertForSequenceClassification.from_pretrained('bert-large-uncased', num_labels=3).to(device)
        elif language_model_to_use == 'roberta-large':
            model = RobertaForSequenceClassification.from_pretrained('roberta-large', num_labels=3).to(device)
        else:
            return -1

    # create train-val split
    train, val = torch.utils.data.random_split(dataset=dataset, lengths=[train_length, val_length])
    dataloaders_dict = {'train': DataLoader(train, batch_size=batch_size, shuffle=True), 'val': DataLoader(val, batch_size=batch_size, shuffle=True)}
    print(train_length, val_length)

    # select optimizer
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    max_num_epochs = 100
    max_early_stopping = 7
    early_stopping_count = 0
    best_ce = float('inf')
    best_accuracy = float('-inf')
    best_f1 = float('-inf')

    eps = 1e-2

    for epoch in range(max_num_epochs):
        if (early_stopping_count >= max_early_stopping):
            break
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
                early_stopping_count += 1
            else:
                model.eval()

            curr_ce = 0
            curr_accuracy = 0
            actual = torch.tensor([]).long().to(device)
            pred = torch.tensor([]).long().to(device)

            for input_ids, attention_masks, labels in dataloaders_dict[phase]:
                input_ids = input_ids.to(device)
                attention_masks = attention_masks.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(input_ids = input_ids, attention_mask = attention_masks, labels=labels)
                    loss = outputs.loss
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    else:
                        curr_ce += loss.item() * input_ids.size(0)
                        curr_accuracy += torch.sum(torch.max(outputs.logits, 1)[1] == labels).item()
                        actual = torch.cat([actual, labels], dim=0)
                        pred= torch.cat([pred, torch.max(outputs.logits, 1)[1]], dim=0)
            if phase == 'val':
                curr_ce = curr_ce / len(val)
                curr_accuracy = curr_accuracy / len(val)
                currF1 = f1_score(actual.cpu().detach().numpy(), pred.cpu().detach().numpy(), average='weighted')
                if curr_ce <= best_ce - eps:
                    best_ce = curr_ce
                    early_stopping_count = 0
                if curr_accuracy >= best_accuracy + eps:
                    best_accuracy = curr_accuracy
                    early_stopping_count = 0
                if currF1 >= best_f1 + eps:
                    best_f1 = currF1
                    early_stopping_count = 0
                print("Val CE: ", curr_ce)
                print("Val Accuracy: ", curr_accuracy)
                print("Val F1: ", currF1)
                print("Early Stopping Count: ", early_stopping_count)

    ## ------------------testing---------------------
    sentence_input_test = []
    labels_output_test = []
    for i, sentence in enumerate(sentences_test):
        if isinstance(sentence, str):
            tokens = tokenizer(sentence)['input_ids']
            sentence_input_test.append(sentence)
            labels_output_test.append(labels_test[i])
        else:
            pass

    tokens_test = tokenizer(sentence_input_test, return_tensors='pt', padding=True, truncation=True, max_length=max_length)
    labels_test = np.array(labels_output_test)

    input_ids_test = tokens_test['input_ids']
    attention_masks_test = tokens_test['attention_mask']
    labels_test = torch.LongTensor(labels_test)
    dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

    dataloaders_dict_test = {'test': DataLoader(dataset_test, batch_size=batch_size, shuffle=True)}
    test_ce = 0
    test_accuracy = 0
    actual = torch.tensor([]).long().to(device)
    pred = torch.tensor([]).long().to(device)
    for input_ids, attention_masks, labels in dataloaders_dict_test['test']:
        input_ids = input_ids.to(device)
        attention_masks = attention_masks.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        with torch.no_grad():
            outputs = model(input_ids = input_ids, attention_mask = attention_masks, labels=labels)
            loss = outputs.loss
            test_ce += loss.item() * input_ids.size(0)
            test_accuracy += torch.sum(torch.max(outputs.logits, 1)[1] == labels).item()
            actual = torch.cat([actual, labels], dim=0)
            pred = torch.cat([pred, torch.max(outputs.logits, 1)[1]], dim=0)
    test_ce = test_ce / len(dataset_test)
    test_accuracy = test_accuracy/ len(dataset_test)
    test_f1 = f1_score(actual.cpu().detach().numpy(), pred.cpu().detach().numpy(), average='weighted')
    experiment_results = [learning_rate, batch_size, best_ce, best_accuracy, best_f1, test_ce, test_accuracy, test_f1]

    # save model
    if save_model_path != None:
        model.save_pretrained(save_model_path)
        tokenizer.save_pretrained(save_model_path)

    return experiment_results

In [4]:
def train_lm_price_change_experiments(gpu_numbers: str,
                                      train_data_path: str,
                                      test_data_path: str,
                                      language_model_to_use: str,
                                      data_category: str):
    """
    Description: Run experiments over different batch sizes, learning rates to find best hyperparameters
    """
    results = []
    batch_sizes = [32, 16]
    learning_rates = [1e-4, 1e-5]
    count = 0
    total_experiments = len(batch_sizes) * len(learning_rates)

    for k, batch_size in enumerate(batch_sizes):
        for j, learning_rate in enumerate(learning_rates):
            count += 1
            print(f'Experiment {count} of {total_experiments}:')

            results.append(train_lm_hawkish_dovish(gpu_numbers,
                                                   train_data_path,
                                                   test_data_path,
                                                   language_model_to_use,
                                                   batch_size,
                                                   learning_rate,
                                                   None))
            df = pd.DataFrame(results, columns=["Learning Rate",
                                                "Batch Size",
                                                "Val Cross Entropy",
                                                "Val Accuracy",
                                                "Val F1 Score",
                                                "Test Cross Entropy",
                                                "Test Accuracy",
                                                "Test F1 Score"])
            df.to_excel(f'grid_search_results/final_{data_category}_{language_model_to_use}.xlsx', index=False)

In [5]:
start_t = time()

# experiments
# roberta-large: https://huggingface.co/FacebookAI/roberta-large, https://arxiv.org/abs/1907.11692
# finbert: https://huggingface.co/yiyanghkust/finbert-tone, https://arxiv.org/abs/2006.08097, https://github.com/yya518/FinBERT
# flangbert: https://huggingface.co/SALT-NLP/FLANG-BERT
# flangroberta: https://huggingface.co/SALT-NLP/FLANG-Roberta
# for language_model_to_use in ["roberta", "roberta-large", "bert", "bert-large", "finbert", "flangbert", "flangroberta"]:
for language_model_to_use in ["roberta", "flangbert", "flangroberta", "finbert"]:
    # for data_category in ["lab-manual-combine", "lab-manual-sp", "lab-manual-mm", "lab-manual-pc", "lab-manual-mm-split", "lab-manual-pc-split", "lab-manual-sp-split", "lab-manual-split-combine"]:
    print("Model name: ", language_model_to_use)
    for data_category in ["lab-manual-combine", "lab-manual-split-combine"]:
        print("Data category: ", data_category)
        train_data_path = "data_PEFT/" + data_category + "-train.xlsx"
        test_data_path = "data_PEFT/" + data_category + "-test.xlsx"
        train_lm_price_change_experiments(gpu_numbers="0",
                                          train_data_path=train_data_path,
                                          test_data_path=test_data_path,
                                          language_model_to_use=language_model_to_use,
                                          data_category=data_category)

print((time() - start_t)/60.0)

Model name:  roberta
Data category:  lab-manual-combine
Experiment 1 of 4:
Device assigned:  cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

Train Size: 4568, Validation Size: 1141


pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

4568 1141
Val CE:  0.5056246235431233
Val Accuracy:  0.8071866783523225
Val F1:  0.8076877107593649
Early Stopping Count:  0
Val CE:  0.2631292907320335
Val Accuracy:  0.9184925503943909
Val F1:  0.9187903889360027
Early Stopping Count:  0
Val CE:  0.25387894813425177
Val Accuracy:  0.9412795793163892
Val F1:  0.9412813947764086
Early Stopping Count:  0
Val CE:  0.2681989641313251
Val Accuracy:  0.9421560035056967
Val F1:  0.9421353838467235
Early Stopping Count:  1
Val CE:  0.3063881880473949
Val Accuracy:  0.9386503067484663
Val F1:  0.9385971144395652
Early Stopping Count:  2
Val CE:  0.33762269671500306
Val Accuracy:  0.9395267309377738
Val F1:  0.9395314743540556
Early Stopping Count:  3
Val CE:  0.3295049249838571
Val Accuracy:  0.9316389132340053
Val F1:  0.9312111714191436
Early Stopping Count:  4
Val CE:  0.284946071602711
Val Accuracy:  0.9430324276950044
Val F1:  0.943130680122113
Early Stopping Count:  5
Val CE:  0.3447395204374604
Val Accuracy:  0.9430324276950044
Val F1: 